In [1]:
from typing import Optional

import ase
from ase.calculators.calculator import Calculator
from sella import Constraints, Internals, Sella


def sella_optimize(
    atms_obj: ase.Atoms,
    calc: Calculator,
    order: int = 0,
    ints_obj: Optional[Internals] = None,
    in_place: bool = False,
) -> ase.Atoms:
    """Optimize a geometry using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param order: 0 = minimum | 1 = saddle point
    :param ints_obj: A Sella Internals object, possibly involving constraints
    :param in_place: Modify the atoms object in place?
    """
    if not in_place:
        atms_obj = atms_obj.copy()

    atms_obj.calc = calc

    # Initialize and run the optimization
    dyn = Sella(
        atms_obj, order=order, internal=(True if ints_obj is None else ints_obj)
    )
    dyn.run()

    return atms_obj


def sella_optimize_minimum(
    atms_obj: ase.Atoms, calc: Calculator, in_place: bool = False
) -> ase.Atoms:
    """Optimize a minimum-energy structure using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param in_place: Modify the atoms object in place?
    """
    return sella_optimize(atms_obj, calc, order=0, in_place=in_place)


def sella_optimize_ts(
    atms_obj: ase.Atoms, calc: Calculator, in_place: bool = False
) -> ase.Atoms:
    """Optimize a TS/saddle-point structure using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param in_place: Modify the atoms object in place?
    """
    return sella_optimize(atms_obj, calc, order=1, in_place=in_place)


def sella_optimize_constrained(
    atms_obj: ase.Atoms,
    calc: Calculator,
    const_coos: Optional[list[tuple[int, ...]]] = None,
    in_place: bool = False,
) -> ase.Atoms:
    """Optimize a structure subject to internal coordinate constraints using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param const_coos: Optionally, constrain a set of coordinates
    :param in_place: Modify the atoms object in place?
    """
    if not in_place:
        atms_obj = atms_obj.copy()

    # Set up constraints
    const_obj = Constraints(atms_obj)
    for coo in const_coos:
        if len(coo) == 2:
            const_obj.fix_bond(coo)
        elif len(coo) == 3:
            const_obj.fix_angle(coo)
        elif len(coo) == 4:
            const_obj.fix_dihedral(coo)

    # Set up internal coordinates
    ints_obj = Internals(atms_obj, cons=const_obj)
    ints_obj.find_all_bonds()
    ints_obj.find_all_angles()
    ints_obj.find_all_dihedrals()

    # Note: For whatever reason, the Atoms object must be identical to the one passed in
    # to Constraints and Internals, so we *must* set in_place=True here
    return sella_optimize(atms_obj, calc, ints_obj=ints_obj, in_place=True)

In [2]:
from tblite.ase import TBLite

calc = TBLite(method="GFN1-xTB", verbosity=0)

file_name = "mol.xyz"
inp_atms_obj = ase.io.read(file_name)

/home/avcopan/code/other/sella-demo/.pixi/envs/default/lib/python3.12/site-packages/tblite/ase.py:39: SyntaxWarning: invalid escape sequence '\ '
  """


In [3]:
min_atms_obj = sella_optimize_minimum(inp_atms_obj.copy(), calc=calc)
ase.io.write(file_name.replace(".xyz", "_min.xyz"), min_atms_obj)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


     Step     Time          Energy         fmax         cmax       rtrust          rho
Sella   0 12:26:19     -168.047759       1.2245       0.0000       0.1000       1.0000


/home/avcopan/code/other/sella-demo/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


Sella   1 12:26:21     -168.069730       0.5523       0.0000       0.1000       0.7088
Sella   2 12:26:21     -168.073385       0.1902       0.0000       0.1000       1.3600
Sella   3 12:26:21     -168.074899       0.0727       0.0000       0.1000       1.2844
Sella   4 12:26:21     -168.075541       0.0884       0.0000       0.1000       1.3959
Sella   5 12:26:21     -168.075956       0.0367       0.0000       0.1000       1.4014


In [4]:
ts_atms_obj = sella_optimize_ts(inp_atms_obj.copy(), calc=calc)
ase.io.write(file_name.replace(".xyz", "_ts.xyz"), ts_atms_obj)

     Step     Time          Energy         fmax         cmax       rtrust          rho
Sella   0 12:26:21     -168.047759       1.2245       0.0000       0.1000       1.0000


/home/avcopan/code/other/sella-demo/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


Sella   1 12:26:22     -168.061872       0.2296       0.0000       0.1000       1.0502
Sella   2 12:26:22     -168.059915       0.1937       0.0000       0.1000       0.6006
Sella   3 12:26:22     -168.054443       0.1672       0.0000       0.1000       0.8271
Sella   4 12:26:22     -168.045858       0.1598       0.0000       0.1000       0.9048
Sella   5 12:26:23     -168.035285       0.1633       0.0000       0.1000       0.9396
Sella   6 12:26:23     -168.023763       0.1526       0.0000       0.1150       0.9854
Sella   7 12:26:23     -168.010935       0.1451       0.0000       0.1322       0.9761
Sella   8 12:26:23     -167.997249       0.1313       0.0000       0.1521       0.9835
Sella   9 12:26:24     -167.983205       0.1137       0.0000       0.1749       0.9868
Sella  10 12:26:24     -167.969647       0.0914       0.0000       0.2011       0.9886
Sella  11 12:26:24     -167.957596       0.0663       0.0000       0.2313       0.9952
Sella  12 12:26:25     -167.947672       0.

In [5]:
const_atms_obj = sella_optimize_constrained(inp_atms_obj.copy(), calc=calc, const_coos=[(1, 3)])
ase.io.write(file_name.replace(".xyz", "_const.xyz"), const_atms_obj)

     Step     Time          Energy         fmax         cmax       rtrust          rho
Sella   0 12:26:25     -168.047759       1.2245       0.0000       0.1000       1.0000
Sella   1 12:26:25     -168.064208       0.2618       0.0000       0.1000       0.8153
Sella   2 12:26:25     -168.065380       0.1148       0.0000       0.1000       1.3584
Sella   3 12:26:25     -168.066236       0.0918       0.0000       0.1000       1.4198
Sella   4 12:26:25     -168.067023       0.1025       0.0000       0.1000       1.3931
Sella   5 12:26:26     -168.067525       0.0641       0.0000       0.1000       1.3565
Sella   6 12:26:26     -168.067757       0.0329       0.0000       0.1000       1.2904
